# 5.3 多输入通道和多输出通道

当输入数据含多个通道时，我们需要构造一个输入通道数与输入数据的通道数相同的卷积核，从而能够与含多通道的输入数据做互相关运算


多输入通道的互相关运算：对每个通道互相关运算，然后使用add_n函数加和，加和之后，输出的通道数就为1。
> 卷积运算中，将每个输入通道与卷积核所有通道进行互相相关运算，将结果进行加和，就得到单通道输出。将所有的单通道输出合并，就得到多输出通道。

代码：

In [1]:
import torch
from torch import nn
import d2l_pytorch as d2l

In [2]:
def corr2d_multi_in(X,K):
    # 沿着X和K的第0维(通道维)卷积，再相加
    # 对第1个通道，进行卷积运算
    res=d2l.corr2d(X[0,:,:],K[0,:,:])
    # 对剩余通道进行卷积运算
    for i in range(1,X.shape[0]):
        res+=d2l.corr2d(X[i,:,:],K[i,:,:])
    return res 

In [3]:
# test
X = torch.tensor([
    [
     [0, 1, 2], 
     [3, 4, 5], 
     [6, 7, 8]
    ],
    [
     [1, 2, 3], 
     [4, 5, 6], 
     [7, 8, 9]
    ]
])

K = torch.tensor([
    [   [0, 1], 
        [2, 3]
    ], 
    [
        [1, 2], 
        [3, 4]
    ]
])

print("X:",X.shape)
print("K:",K.shape)


corr2d_multi_in(X, K)

X: torch.Size([2, 3, 3])
K: torch.Size([2, 2, 2])


tensor([[ 56.,  72.],
        [104., 120.]])

## 多输出通道

代码：

In [4]:
def corr2d_multi_out(X,K):
    # 对K的所有维进行遍历，每次遍历：该维与X做互相关运算，这样就能得到一个通道的输出了
    # 最后将结果合并在一起
    return torch.stack([corr2d_multi_in(X,k) for k in K])

In [5]:
# test
# 2*2*2的卷积核，最左边的2表示通道数
K = torch.tensor([
    [   [0, 1], 
        [2, 3]
    ], 
    [
        [1, 2], 
        [3, 4]
    ]
])
# 2*2*2,2个通道数,一共3个批量
K=torch.stack([K,K+1,K+2])
print("K:",K.shape)
print("X:",X.shape)
print(K)
# batch_size, channel, h, w

K: torch.Size([3, 2, 2, 2])
X: torch.Size([2, 3, 3])
tensor([[[[0, 1],
          [2, 3]],

         [[1, 2],
          [3, 4]]],


        [[[1, 2],
          [3, 4]],

         [[2, 3],
          [4, 5]]],


        [[[2, 3],
          [4, 5]],

         [[3, 4],
          [5, 6]]]])


In [6]:
Y=corr2d_multi_out(X,K)
print('Y:',Y.shape)
Y

Y: torch.Size([3, 2, 2])


tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [7]:
print(X.shape)

torch.Size([2, 3, 3])


## 1\*1的卷积层

卷积窗口长和宽都为1的多通道卷积层称为1*1卷积层，

该层输入和输出具有相同的高和宽，输出中的每个元素来自输入中在高和宽上相同位置的元素在不同通道之间的按权重累加。

假设我们将通道维当作特征维，将高和宽维度上的元素当成数据样本，那么1×1卷积层的作用与全连接层等价。

经常当作保持高和宽维度形状不变的全连接层使用。

In [8]:
def corr2d_multi_out_1(X,K):
    c_i,h,w=X.shape
    c_o=K.shape[0]
    # 改变输入的维度
    X=X.view(c_i,h*w)
    # 改变卷积层的维度
    K=K.view(c_o,c_i)
    # 输出
    return torch.mm(K,X).view(c_o,h,w)

In [9]:
# test
X = torch.rand(3, 3, 3)
K = torch.rand(2, 3, 1, 1)

Y1=corr2d_multi_out(X,K)
Y2=corr2d_multi_out_1(X,K)

(Y1-Y2).norm().item()<1e-6

True

# 5.4 池化层

池化层的提出，是为了缓解卷积层对于位置的过度敏感性。

分类：

+ 最大池化

+ 平均池化

代码：

In [10]:
def pool2d(X,pool_size,mode='max'):
    X=X.float()
    # 池化层的高和宽
    p_h,p_w=pool_size
    # 输出，尺寸
    Y=torch.zeros(X.shape[0]-p_h+1,X.shape[1]-p_w+1)
    # 行
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode=='max':
                Y[i,j]=X[i:i+p_h,j:j+p_w].max()
            elif mode=='avg':
                Y[i,j]=X[i:i+p_h,j:j+p_w].mean()
    return Y

In [11]:
# test
X=torch.tensor([
    [0,1,2],
    [3,4,5],
    [6,7,8]
])
pool2d(X,(2,2))

tensor([[4., 5.],
        [7., 8.]])

In [12]:
pool2d(X,(2,2),'avg')

tensor([[2., 3.],
        [5., 6.]])

## 池化层中使用填充和步幅

简洁实现：使用torch.nn.MaxPool2d()

代码：

In [13]:
X=torch.arange(16,dtype=torch.float)
X=X.view(1,1,4,4)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [14]:
# 使用形状为3*3的池化窗口，步幅默认为3
pool2d=torch.nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

In [15]:
# 指定步幅和填充

pool2d=torch.nn.MaxPool2d(3,padding=1,stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [16]:
pool2d = torch.nn.MaxPool2d((2, 4), padding=(1, 2), stride=(2, 3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

## 多通道的池化层

在处理多通道输入数据时，池化层对每个输入通道分别池化，而不是像卷积层那样将各通道的输入按通道相加。这意味着池化层的输出通道数与输入通道数相等

In [20]:
# X2=torch.stack([X,X+1]) 
# 不等价于
X2=torch.cat((X,X+1),dim=1)
X2

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [28]:
print(X.shape)
print(X2.shape)

torch.Size([1, 1, 4, 4])
torch.Size([1, 2, 4, 4])


In [30]:
X3=torch.stack([X,X+1],dim=1)
print(X3.shape)
print(X3)

torch.Size([1, 2, 1, 4, 4])
tensor([[[[[ 0.,  1.,  2.,  3.],
           [ 4.,  5.,  6.,  7.],
           [ 8.,  9., 10., 11.],
           [12., 13., 14., 15.]]],


         [[[ 1.,  2.,  3.,  4.],
           [ 5.,  6.,  7.,  8.],
           [ 9., 10., 11., 12.],
           [13., 14., 15., 16.]]]]])


In [31]:
# test

pool2d = torch.nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X2)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])